In [11]:
import os
import random
import numpy as np
import torch

# Please make sure you are using CUDA enabled GPU for this project
device = 'cpu'

# Setting the seed value ensures that the results are reproducible across different runs
seed_val = 10

# Ensuring that the seed is set for Python's hashing, random operations, NumPy, and PyTorch
os.environ['PYTHONHASHSEED'] = str(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
llh_shift = torch.tensor(5.0)

In [ ]:
def compute_log_likelihood_metrics(results_list):
    """
    Compute log likelihood and related metrics for all generations under their given context.
    
    results_list: list of tuples with model size and result dictionary
    
    returns: dictionary with keys such as 'neg_log_likelihoods', 'average_neg_log_likelihoods', etc.
             containing tensors of shape (num_models, num_generations, num_samples_per_generation)
    """

    # Initialize a dictionary to store the results
    metrics_dict = {}

    # Define the keys that need to be processed
    keys_to_process = ['neg_log_likelihoods', 'average_neg_log_likelihoods', 'sequence_embeddings',
                       'pointwise_mutual_information', 'average_neg_log_likelihood_of_most_likely_gen',
                       'average_neg_log_likelihood_of_second_most_likely_gen', 'neg_log_likelihood_of_most_likely_gen', 'semantic_set_ids']

    # Iterate through the keys and process the results
    for key in keys_to_process:
        ids_list = []
        combined_results = []
        for model_size, result in results_list:
            results_for_model = []
            for sample in result:
                avg_neg_log_likelihoods = sample[key]
                ids_list.append(sample['id'])
                results_for_model.append(avg_neg_log_likelihoods)

            results_for_model = torch.stack(results_for_model)
            combined_results.append(results_for_model)

        # Stack the results if the key is not 'sequence_embeddings'
        if key != 'sequence_embeddings':
            combined_results = torch.stack(combined_results)

        metrics_dict[key] = combined_results

    # Add the list of IDs to the result dictionary
    metrics_dict['id'] = ids_list

    return metrics_dict

def compute_mutual_information(log_likelihoods):
    """Compute a confidence measure for a given set of likelihoods using mutual information."""
    
    num_models = torch.tensor(log_likelihoods.shape[0])
    mean_across_models = torch.logsumexp(log_likelihoods, dim=0) - torch.log(num_models)
    tiled_mean = mean_across_models.tile(num_models, 1, 1)
    diff_term = torch.exp(log_likelihoods) * log_likelihoods - torch.exp(tiled_mean) * tiled_mean
    f_j = torch.div(torch.sum(diff_term, dim=0), diff_term.shape[0])
    mutual_information = torch.div(torch.sum(torch.div(f_j, mean_across_models), dim=1), f_j.shape[-1])

    return mutual_information

def compute_log_likelihood_variance(neg_log_likelihoods):
    """Compute the variance of negative log likelihoods for approximate posterior predictive."""
    
    mean_across_models = torch.mean(neg_log_likelihoods, dim=0)
    variance_of_neg_log_likelihoods = torch.var(mean_across_models, dim=1)

    return variance_of_neg_log_likelihoods

def compute_log_likelihood_mean(neg_log_likelihoods):
    """Compute the mean of negative log likelihoods for approximate posterior predictive."""
    
    mean_across_models = torch.mean(neg_log_likelihoods, dim=0)
    mean_of_neg_log_likelihoods = torch.mean(mean_across_models, dim=1)

    return mean_of_neg_log_likelihoods

def compute_mean_pointwise_mutual_information(pointwise_mutual_information):
    """Compute the mean of pointwise mutual information across models."""
    
    mean_across_models = torch.mean(pointwise_mutual_information, dim=0)
    return torch.mean(mean_across_models, dim=1)

def compute_predictive_entropy(log_likelihoods):
    """Compute the predictive entropy of approximate posterior predictive."""
    
    num_models = torch.tensor(log_likelihoods.shape[0])
    mean_across_models = torch.logsumexp(log_likelihoods, dim=0) - torch.log(num_models)
    entropy = -torch.sum(mean_across_models, dim=1) / torch.tensor(mean_across_models.shape[1])

    return entropy

def compute_predictive_entropy_across_concepts(log_likelihoods, semantic_set_ids):
    """Compute the semantic entropy across different concepts."""
    
    num_models = torch.tensor(log_likelihoods.shape[0])
    mean_across_models = torch.logsumexp(log_likelihoods, dim=0) - torch.log(num_models)
    semantic_set_ids = semantic_set_ids[0]  # All models have the same semantic set ids
    entropies = []
    for row_index in range(mean_across_models.shape[0]):
        aggregated_likelihoods = []
        row = mean_across_models[row_index]
        semantic_ids_row = semantic_set_ids[row_index].to(device)
        for semantic_set_id in torch.unique(semantic_ids_row):
            aggregated_likelihoods.append(torch.logsumexp(row[semantic_ids_row == semantic_set_id], dim=0))
        aggregated_likelihoods = torch.tensor(aggregated_likelihoods) - llh_shift
        entropy = - torch.sum(aggregated_likelihoods, dim=0) / torch.tensor(aggregated_likelihoods.shape[0])
        entropies.append(entropy)

    return torch.tensor(entropies)

def compute_margin_probability_uncertainty(log_likelihoods):
    """Compute margin probability uncertainty measure."""
    
    num_models = torch.tensor(log_likelihoods.shape[0])
    mean_across_models = torch.logsumexp(log_likelihoods, dim=0) - torch.log(num_models)
    topk_likelihoods, _ = torch.topk(mean_across_models, 2, dim=1, sorted=True)
    margin_probabilities = np.exp(topk_likelihoods[:, 0]) - np.exp(topk_likelihoods[:, 1])

    return margin_probabilities

In [13]:
import pickle

list_of_results = []

with open('data/likelihoods_st.pkl', 'rb') as infile:
    sequences = pickle.load(infile)
    list_of_results.append(('gemma', sequences))

In [14]:
# Compute overall log likelihoods and related metrics
log_likelihood_metrics = compute_log_likelihood_metrics(list_of_results)

# Compute mutual information from negative log likelihoods
mutual_information = compute_mutual_information(-log_likelihood_metrics['neg_log_likelihoods'])

# Compute predictive entropy from negative log likelihoods
predictive_entropy = compute_predictive_entropy(-log_likelihood_metrics['neg_log_likelihoods'])

# Compute predictive entropy over different semantic concepts
entropy_across_concepts = compute_predictive_entropy_across_concepts(-log_likelihood_metrics['average_neg_log_likelihoods'],
                                                                     log_likelihood_metrics['semantic_set_ids'])

# Compute unnormalized entropy over different semantic concepts
unnormalized_entropy_across_concepts = compute_predictive_entropy_across_concepts(-log_likelihood_metrics['neg_log_likelihoods'],
                                                                                  log_likelihood_metrics['semantic_set_ids'])

# Compute margin probability uncertainty measures
margin_probabilities = compute_margin_probability_uncertainty(-log_likelihood_metrics['average_neg_log_likelihoods'])
unnormalized_margin_probabilities = compute_margin_probability_uncertainty(-log_likelihood_metrics['neg_log_likelihoods'])

In [15]:
def count_unique_elements_per_row(tensor):
    """Count the number of unique elements in each row of a tensor."""
    assert len(tensor.shape) == 2
    return torch.count_nonzero(torch.sum(torch.nn.functional.one_hot(tensor), dim=1), dim=1)


In [16]:
# Count the number of unique semantic sets
num_semantic_sets = count_unique_elements_per_row(log_likelihood_metrics['semantic_set_ids'][0])

# Compute average predictive entropy
avg_predictive_entropy = compute_predictive_entropy(-log_likelihood_metrics['average_neg_log_likelihoods'])

# Initialize lists to store entropy measures on subsets
avg_entropy_on_subsets = []
entropy_on_subsets = []
semantic_entropy_on_subsets = []
num_semantic_sets_on_subsets = []
num_predictions = log_likelihood_metrics['average_neg_log_likelihoods'].shape[-1]

# Compute entropy measures on subsets of predictions
for i in range(1, num_predictions + 1):
    offset = num_predictions * (i / 100)
    avg_entropy_on_subsets.append(compute_predictive_entropy(-log_likelihood_metrics['average_neg_log_likelihoods'][:, :, :int(i)]))
    entropy_on_subsets.append(compute_predictive_entropy(-log_likelihood_metrics['neg_log_likelihoods'][:, :, :int(i)]))
    semantic_entropy_on_subsets.append(compute_predictive_entropy_across_concepts(-log_likelihood_metrics['average_neg_log_likelihoods'][:, :, :int(i)],
                                                                                  log_likelihood_metrics['semantic_set_ids'][:, :, :int(i)]))
    num_semantic_sets_on_subsets.append(count_unique_elements_per_row(log_likelihood_metrics['semantic_set_ids'][0][:, :i]))

# Compute average pointwise mutual information
avg_pointwise_mutual_info = compute_mean_pointwise_mutual_information(log_likelihood_metrics['pointwise_mutual_information'])

In [17]:
# Adding computed metrics to the overall results dictionary
log_likelihood_metrics['mutual_information'] = mutual_information
log_likelihood_metrics['predictive_entropy'] = predictive_entropy
log_likelihood_metrics['entropy_across_concepts'] = entropy_across_concepts
log_likelihood_metrics['unnormalized_entropy_across_concepts'] = unnormalized_entropy_across_concepts
log_likelihood_metrics['num_semantic_sets'] = num_semantic_sets
log_likelihood_metrics['margin_probabilities'] = margin_probabilities
log_likelihood_metrics['unnormalized_margin_probabilities'] = unnormalized_margin_probabilities
log_likelihood_metrics['avg_predictive_entropy'] = avg_predictive_entropy

# Adding computed metrics on subsets to the overall results dictionary
for i in range(len(avg_entropy_on_subsets)):
    log_likelihood_metrics[f'avg_entropy_on_subset_{i + 1}'] = avg_entropy_on_subsets[i]
    log_likelihood_metrics[f'entropy_on_subset_{i + 1}'] = entropy_on_subsets[i]
    log_likelihood_metrics[f'semantic_entropy_on_subset_{i + 1}'] = semantic_entropy_on_subsets[i]
    log_likelihood_metrics[f'num_semantic_sets_on_subset_{i + 1}'] = num_semantic_sets_on_subsets[i]

log_likelihood_metrics['avg_pointwise_mutual_info'] = avg_pointwise_mutual_info


In [18]:
from pprint import pprint
pprint(log_likelihood_metrics.keys())

dict_keys(['neg_log_likelihoods', 'average_neg_log_likelihoods', 'sequence_embeddings', 'pointwise_mutual_information', 'average_neg_log_likelihood_of_most_likely_gen', 'average_neg_log_likelihood_of_second_most_likely_gen', 'neg_log_likelihood_of_most_likely_gen', 'semantic_set_ids', 'id', 'mutual_information', 'predictive_entropy', 'entropy_across_concepts', 'unnormalized_entropy_across_concepts', 'num_semantic_sets', 'margin_probabilities', 'unnormalized_margin_probabilities', 'avg_predictive_entropy', 'avg_entropy_on_subset_1', 'entropy_on_subset_1', 'semantic_entropy_on_subset_1', 'num_semantic_sets_on_subset_1', 'avg_entropy_on_subset_2', 'entropy_on_subset_2', 'semantic_entropy_on_subset_2', 'num_semantic_sets_on_subset_2', 'avg_entropy_on_subset_3', 'entropy_on_subset_3', 'semantic_entropy_on_subset_3', 'num_semantic_sets_on_subset_3', 'avg_entropy_on_subset_4', 'entropy_on_subset_4', 'semantic_entropy_on_subset_4', 'num_semantic_sets_on_subset_4', 'avg_entropy_on_subset_5', 'e

In [19]:
with open(f'data/aggregated_likelihoods_generations_st.pkl', 'wb') as outfile:
    pickle.dump(log_likelihood_metrics, outfile)

In [20]:
print('Margin probabilities:', margin_probabilities)
print('Number of semantic sets:', num_semantic_sets)
print('predictive entropy shape:', predictive_entropy.shape)
print('predictive entropy per concept shape:', entropy_across_concepts.shape)
print(log_likelihood_metrics['average_neg_log_likelihoods'].shape)
print(len(num_semantic_sets_on_subsets))
print(num_semantic_sets_on_subsets[0].shape)
print('average predictive entropy on subsets:', len(avg_entropy_on_subsets))
print(avg_entropy_on_subsets[0].shape)
print(log_likelihood_metrics['pointwise_mutual_information'])
print(log_likelihood_metrics['margin_probabilities'])

Margin probabilities: tensor([6.5584e-02, 4.4359e-02, 1.4562e-02, 8.6575e-02, 2.0068e-03, 5.8177e-02,
        7.2310e-02, 8.6190e-02, 3.8222e-02, 7.4364e-02, 1.2002e-01, 1.7283e-01,
        9.3625e-03, 1.9129e-01, 3.0171e-01, 6.9890e-02, 2.2424e-01, 2.5570e-02,
        5.4232e-02, 1.4635e-01, 1.6209e-02, 6.7409e-02, 1.9434e-01, 4.0876e-01,
        1.7213e-01, 5.5735e-02, 3.9214e-02, 1.6821e-01, 1.0261e-01, 4.2534e-03,
        1.7914e-01, 5.3702e-03, 1.5029e-02, 2.4892e-02, 2.0591e-01, 4.4349e-02,
        6.3916e-02, 7.1394e-03, 2.6645e-03, 1.0954e-02, 4.4994e-01, 6.6210e-02,
        2.9049e-02, 1.8406e-01, 3.7423e-02, 1.8155e-01, 1.1209e-02, 5.8704e-02,
        1.8377e-01, 1.8573e-01, 1.1689e-02, 3.5077e-02, 1.7494e-01, 1.6816e-02,
        1.6451e-02, 3.2596e-02, 8.6117e-03, 9.6717e-02, 5.1238e-02, 1.9429e-03,
        7.0909e-02, 1.2721e-03, 5.5869e-02, 1.8749e-02, 1.1596e-02, 4.5479e-02,
        1.6516e-02, 1.0492e-02, 2.5851e-03, 1.2103e-02, 4.7382e-03, 1.3390e-02,
        1.0201e-02